In [1]:
!pip install lightgbm
#Ensemble boositng works best with lightgbm, the gbm means gradient boosting model, 
#Remember to make notes on this when the competition ends

In [4]:
import numpy as np
import pandas as pd

from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold,StratifiedKFold ,GroupKFold
from sklearn.metrics import mean_squared_error


import warnings
warnings.filterwarnings('ignore')

In [13]:
#Python, unlike Java, favors me when it comes to functions, so we'll create on that collects data from both sets(train and test)
def get_processed_data(train_path,test_path) :

    #Import the data
    
    train = pd.read_csv(r'C:\Users\shee\Desktop\Umoja Hack Zindi\economic-well-being-prediction-competition\Train.csv')
    test  = pd.read_csv(r'C:\Users\shee\Desktop\Umoja Hack Zindi\economic-well-being-prediction-competition\Test.csv')

    data = pd.concat([train, test]).reset_index(drop=True)
    
#These are the unique columsn to predict the wealth
    col = ['country', 'year', 'urban_or_rural']
    
    ## Count of unique features
    for i in col:
        data['count_'+i] = data[i].map(data[i].value_counts())
        
    ## Combination features
    data['all_ghsl'] = data['ghsl_built_1975_to_1990']+data['ghsl_built_pre_1975']+data['ghsl_built_1990_to_2000']+data['ghsl_built_2000_to_2014']
    data['all_landcover_fraction'] = data['landcover_crops_fraction']+data['landcover_urban_fraction']
    data['all_waters'] = data['landcover_water_permanent_10km_fraction'] + data['landcover_water_seasonal_10km_fraction']
    
    # get train , test
    train = data[data['ID'].isin(train['ID'].values)]
    test = data[~data['ID'].isin(train['ID'].values)]
    #Don't forget our unique columsn
    features = [x for x in train.columns if x not in 
                ['ID','country','urban_or_rural','Target','year']]
    return train , test , features

In [14]:
train_data = 'Train.csv' ; test_data = 'Test.csv'
train , test , features = get_processed_data(train_path,test_path)

In [15]:
#This function will be our magic model. I'm hoping the predictions wouldn't be as worse than the one's I've handed in so far.
def get_model(Name='lgbm') :
    if Name=='lgbm' :
      return LGBMRegressor(**{'objective' :'regression','boosting_type' : 'gbdt','metric': 'rmse' ,
                              'learning_rate' : 0.05,'num_iterations': 1500,'max_depth' :4 ,'num_leaves' : 150,
                              'max_bins': 85,'min_data_in_leaf':30,'reg_lambda' :75})

In [16]:
Model_Name = "lgbm"

X = train[features]
y = train['Target']
test_ = test[features]

folds = KFold(n_splits=10, shuffle=True, random_state=2021)
oofs  = np.zeros((len(X)))
test_predictions = np.zeros((len(test)))


for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y)):
    
    X_trn, y_trn = X.iloc[trn_idx], y.iloc[trn_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
    

    clf = get_model(Name=Model_Name)
    clf.fit(X_trn, y_trn, eval_set = [(X_val, y_val)],
            verbose = 0, early_stopping_rounds = 50)
    
    vp = clf.predict(X_val)
    oofs[val_idx] = vp
    val_score = mean_squared_error((vp), (y_val),squared=False)
    print(4*'-- -- -- --')
    print(f'Fold {fold_+1} Val score: {val_score}')
    print(4*'-- -- -- --')
    
    tp = clf.predict(test_)
    test_predictions += tp / folds.n_splits

print()
print(3*'###',10*"^",3*'###')
print(mean_squared_error(y, oofs,squared=False))


[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
-- -- -- ---- -- -- ---- -- -- ---- -- -- --
Fold 1 Val score: 0.08752408256678368
-- -- -- ---- -- -- ---- -- -- ---- -- -- --
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
-- -- -- ---- -- -- ---- -- -- ---- -- -- --
Fold 2 Val score: 0.09237492907308259
-- -- -- ---- -- -- ---- -- -- ---- -- -- --
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
-- -- -- ---- -- -- ---- -- -- ---- -- -- --
Fold 3 Val score: 0.08227360283659732
-- -- -- ---- -- -- ---- -- -- ---- -- -- --
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
-- -- -- ---- -- -- ---- -- -- ---- -- -- --
Fold 4 Val score: 0.08561543523309978
-- -- -- ---- -- -- ---- -- -- ---- -- -- --


In [17]:
submission = pd.DataFrame()
submission['ID'] = test['ID']
submission['Target'] = np.clip(test_predictions, 0.141000, 0.808657)

In [20]:
submission.to_csv(r'C:\Users\shee\Desktop\Umoja Hack Zindi\economic-well-being-prediction-competition\first_submission.csv', index=False)